In [1]:
!pip install visualkeras

In [2]:
import os
import numpy as np
import shutil
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import cv2
from torchvision.transforms.functional import to_pil_image
from collections import defaultdict
from torchvision.datasets import ImageFolder
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.optim as optim
import torch.nn.functional as F
from torchvision.transforms import ToPILImage
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from keras.utils import to_categorical
from keras.models import Model,Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.applications.vgg16 import preprocess_input,VGG16
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint,EarlyStopping
import visualkeras
from PIL import ImageFont

In [3]:
import random

seed_constant = 10
random.seed(seed_constant)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#!wget -O imdb_crop.tar http://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar


In [6]:
#!tar -xvf imdb_crop.tar

In [7]:
from scipy.io import loadmat

# Load the metadata
metadata_path = "/content/imdb_crop/imdb.mat"
mat_data = loadmat(metadata_path)

# Print all available keys in the .mat file
print(mat_data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'imdb'])


In [8]:
# Inspect the structure of 'imdb'
import numpy as np
imdb_data = mat_data["imdb"]  # Extract the 'imdb' key
print(type(imdb_data))        # Check the type of imdb_data
print(imdb_data.shape)        # Check the shape
print(imdb_data.dtype)        # Check the dtype

# Explore the first entry of imdb_data to see its format
print(imdb_data[0][0])        # The structure of the 'imdb' object

<class 'numpy.ndarray'>
(1, 1)
[('dob', 'O'), ('photo_taken', 'O'), ('full_path', 'O'), ('gender', 'O'), ('name', 'O'), ('face_location', 'O'), ('face_score', 'O'), ('second_face_score', 'O'), ('celeb_names', 'O'), ('celeb_id', 'O')]
(array([[693726, 693726, 693726, ..., 726831, 726831, 726831]], dtype=int32), array([[1968, 1970, 1968, ..., 2011, 2011, 2011]], dtype=uint16), array([[array(['01/nm0000001_rm124825600_1899-5-10_1968.jpg'], dtype='<U43'),
        array(['01/nm0000001_rm3343756032_1899-5-10_1970.jpg'], dtype='<U44'),
        array(['01/nm0000001_rm577153792_1899-5-10_1968.jpg'], dtype='<U43'),
        ...,
        array(['08/nm3994408_rm926592512_1989-12-29_2011.jpg'], dtype='<U44'),
        array(['08/nm3994408_rm943369728_1989-12-29_2011.jpg'], dtype='<U44'),
        array(['08/nm3994408_rm976924160_1989-12-29_2011.jpg'], dtype='<U44')]],
      dtype=object), array([[1., 1., 1., ..., 0., 0., 0.]]), array([[array(['Fred Astaire'], dtype='<U12'),
        array(['Fred Astair

In [9]:
# Access the nested structure within 'imdb'
imdb_data = mat_data["imdb"][0][0]

# Extract each field
dob = imdb_data["dob"][0]              # Date of birth
photo_taken = imdb_data["photo_taken"][0]  # Year photo was taken
full_paths = imdb_data["full_path"][0]     # File paths to images
names = imdb_data["name"][0]               # Names of actors/actresses
face_scores = imdb_data["face_score"][0]   # Face detection scores
celeb_ids = imdb_data["celeb_id"][0]       # Celebrity IDs
face_locations = imdb_data["face_location"][0]  # Face locations

# Print examples
print("Date of Birth Example:", dob[:5])
print("Photo Taken Example:", photo_taken[:5])
print("Full Paths Example:", full_paths[:5])
print("Names Example:", names[:5])
print("Face Scores Example:", face_scores[:5])
print("Celebrity IDs Example:", celeb_ids[:5])
print("Face Location Example:", face_locations[:5])

Date of Birth Example: [693726 693726 693726 693726 693726]
Photo Taken Example: [1968 1970 1968 1968 1968]
Full Paths Example: [array(['01/nm0000001_rm124825600_1899-5-10_1968.jpg'], dtype='<U43')
 array(['01/nm0000001_rm3343756032_1899-5-10_1970.jpg'], dtype='<U44')
 array(['01/nm0000001_rm577153792_1899-5-10_1968.jpg'], dtype='<U43')
 array(['01/nm0000001_rm946909184_1899-5-10_1968.jpg'], dtype='<U43')
 array(['01/nm0000001_rm980463616_1899-5-10_1968.jpg'], dtype='<U43')]
Names Example: [array(['Fred Astaire'], dtype='<U12')
 array(['Fred Astaire'], dtype='<U12')
 array(['Fred Astaire'], dtype='<U12')
 array(['Fred Astaire'], dtype='<U12')
 array(['Fred Astaire'], dtype='<U12')]
Face Scores Example: [1.45969291 2.5431976  3.45557949 1.87211717 1.15876579]
Celebrity IDs Example: [6488 6488 6488 6488 6488]
Face Location Example: [array([[1072.926,  161.838, 1214.784,  303.696]])
 array([[477.184, 100.352, 622.592, 245.76 ]])
 array([[114.96964309, 114.96964309, 451.68657236, 451.68657

In [10]:
import datetime

# Convert Matlab serial dates to years for birth years
def matlab_datenum_to_datetime(datenum):
    matlab_epoch = datetime.datetime(1, 1, 1)
    return matlab_epoch + datetime.timedelta(days=int(datenum) - 1)

dob_years = [matlab_datenum_to_datetime(d).year if not np.isnan(d) else None for d in dob]

# Calculate ages and filter valid samples
ages = [photo - dob if dob is not None else None for photo, dob in zip(photo_taken, dob_years)]
valid_indices = [i for i, age in enumerate(ages) if age is not None and age >= 0]

# Filter valid data
filtered_paths = [full_paths[i][0] for i in valid_indices]
filtered_ages = [ages[i] for i in valid_indices]

In [11]:
# Print a few examples of filtered paths and ages
print("Example Path and Age Pairs:")
for i, (path, age) in enumerate(zip(filtered_paths, filtered_ages)):
    print(f"Path: {path}, Age: {age}")
    if i == 20:
        break

Example Path and Age Pairs:
Path: 01/nm0000001_rm124825600_1899-5-10_1968.jpg, Age: 68
Path: 01/nm0000001_rm3343756032_1899-5-10_1970.jpg, Age: 70
Path: 01/nm0000001_rm577153792_1899-5-10_1968.jpg, Age: 68
Path: 01/nm0000001_rm946909184_1899-5-10_1968.jpg, Age: 68
Path: 01/nm0000001_rm980463616_1899-5-10_1968.jpg, Age: 68
Path: 02/nm0000002_rm1075631616_1924-9-16_1991.jpg, Age: 66
Path: 02/nm0000002_rm1346607872_1924-9-16_2004.jpg, Age: 79
Path: 02/nm0000002_rm1363385088_1924-9-16_2004.jpg, Age: 79
Path: 02/nm0000002_rm1411175936_1924-9-16_1991.jpg, Age: 66
Path: 02/nm0000002_rm1447271168_1924-9-16_2004.jpg, Age: 79
Path: 02/nm0000002_rm1624085760_1924-9-16_2008.jpg, Age: 83
Path: 02/nm0000002_rm1646056960_1924-9-16_1991.jpg, Age: 66
Path: 02/nm0000002_rm221957120_1924-9-16_1974.jpg, Age: 49
Path: 02/nm0000002_rm2287049216_1924-9-16_2007.jpg, Age: 82
Path: 02/nm0000002_rm238734336_1924-9-16_1974.jpg, Age: 49
Path: 02/nm0000002_rm2585828096_1924-9-16_2006.jpg, Age: 81
Path: 02/nm0000002

In [12]:
# Dataset directory
img_dir = "/content/imdb_crop"

# Data Preprocessing

In [13]:
img_height=224
img_width=224

In [14]:
def label_extraction(age):
    '''
    Categorize age into 5 classes:
    0-20 --> 0, 21-40 --> 1, 41-60 --> 2, 61-80 --> 3, 81-100 --> 4
    '''
    if 0 <= age <= 20:
        return 0
    elif 21 <= age <= 40:
        return 1
    elif 41 <= age <= 60:
        return 2
    elif 61 <= age <= 80:
        return 3
    else:
        return None  # Handles cases outside the range, if necessary

In [15]:
# Image preprocessing function
def image_preprocessing(img_path):
    '''
    Reads, resizes, and normalizes the image.
    '''
    # Read the image
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found or invalid at path: {img_path}")

    # Resize and normalize
    resized_img = cv2.resize(img, (img_height, img_width))
    normalized_img = resized_img / 255.0

    return normalized_img

In [16]:
def balanced_subset(filtered_paths, filtered_ages, subset_size, label_func, num_labels=5):
    # Categorize data by labels
    label_groups = defaultdict(list)
    for path, age in zip(filtered_paths, filtered_ages):
        label = label_func(age)
        if label is not None:
            label_groups[label].append((path, age))

    # Determine the subset size per label
    subset_per_label = subset_size // num_labels

    # Select equal samples for each label
    filtered_paths_subset = []
    filtered_ages_subset = []

    for label in range(num_labels):
        if label in label_groups:
            samples = label_groups[label]
            selected_samples = samples[:subset_per_label]  # Take the first N samples
            filtered_paths_subset.extend([s[0] for s in selected_samples])
            filtered_ages_subset.extend([s[1] for s in selected_samples])

    # Combine paths and ages into the reduced dataset
    return filtered_paths_subset, filtered_ages_subset

In [17]:
# Dataset creation function
def create_dataset(img_paths, img_ages):
    '''
    Preprocesses images and extracts labels for the dataset.
    '''
    features, labels = [], []

    for img_path, age in zip(img_paths, img_ages):
        # Construct full path
        full_img_path = os.path.join(img_dir, img_path)

        # Preprocess image and get class label
        try:
            preprocessed_img = image_preprocessing(full_img_path)
            class_index = label_extraction(age)
        except Exception as e:
            print(f"Skipping image {img_path}: {e}")
            continue

        features.append(preprocessed_img)
        labels.append(class_index)

    return np.asarray(features), np.asarray(labels)

In [18]:
# Reduce dataset size by selecting a random subset of indices
subset_size = 16000
filtered_paths_subset, filtered_ages_subset = balanced_subset(
    filtered_paths, filtered_ages, subset_size, label_extraction
)

# Optionally shuffle the subset for randomness
shuffled_indices = np.random.permutation(len(filtered_paths_subset))
filtered_paths_subset = [filtered_paths_subset[i] for i in shuffled_indices]
filtered_ages_subset = [filtered_ages_subset[i] for i in shuffled_indices]

# Create the reduced dataset
features, labels = create_dataset(filtered_paths_subset, filtered_ages_subset)

In [19]:
# Print a few examples of the filtered dataset
print("Example Dataset Samples:")
for i, (path, age) in enumerate(zip(filtered_paths_subset, filtered_ages_subset)):
    print(f"Path: {path}, Age: {age}, Label: {label_extraction(age)}")
    if i == 20:
        break

Example Dataset Samples:
Path: 42/nm0000142_rm148670976_1930-5-31_2008.jpg, Age: 77, Label: 3
Path: 04/nm0000704_rm3038878464_1981-1-28_2001.jpg, Age: 19, Label: 0
Path: 06/nm0000106_rm1899874304_1975-2-22_2014.jpg, Age: 38, Label: 1
Path: 13/nm0000213_rm3710160384_1971-10-29_1992.jpg, Age: 20, Label: 0
Path: 12/nm0000112_rm3886334208_1953-5-16_2014.jpg, Age: 60, Label: 2
Path: 02/nm0000102_rm643291648_1958-7-8_2013.jpg, Age: 54, Label: 2
Path: 06/nm0000106_rm3418459648_1975-2-22_2009.jpg, Age: 33, Label: 1
Path: 05/nm0000105_rm3609434624_1945-6-11_2005.jpg, Age: 59, Label: 2
Path: 38/nm0001138_rm4066691072_1952-12-10_1970.jpg, Age: 17, Label: 0
Path: 98/nm0000098_rm1163762432_1969-2-11_2005.jpg, Age: 35, Label: 1
Path: 02/nm0000002_rm289065984_1924-9-16_1974.jpg, Age: 49, Label: 2
Path: 64/nm0000164_rm2390858496_1937-12-31_2007.jpg, Age: 68, Label: 3
Path: 63/nm0000663_rm2759170560_1980-8-12_2000.jpg, Age: 19, Label: 0
Path: 93/nm0000093_rm157656832_1963-12-18_2013.jpg, Age: 49, Label

In [20]:
from collections import Counter

# Extract labels for the subset
subset_labels = [label_extraction(age) for age in filtered_ages_subset]

# Count occurrences of each label
label_counts = Counter(subset_labels)

# Print the counts for each label
print("Label Counts in the Dataset:")
for label, count in sorted(label_counts.items()):
    print(f"Label {label}: {count}")

Label Counts in the Dataset:
Label 0: 3200
Label 1: 3200
Label 2: 3200
Label 3: 3200


In [21]:
# One-hot encode the labels
from tensorflow.keras.utils import to_categorical
encoded_labels = to_categorical(labels)

In [22]:
from sklearn.model_selection import train_test_split

# Split into train and test sets with stratification
features_train, features_test, labels_train, labels_test = train_test_split(
    features,
    encoded_labels,
    test_size=0.1,
    shuffle=True,
    random_state=seed_constant,
    stratify=labels  # Ensure equal distribution of labels
)

In [23]:
# Print dataset information
print(f"Training features shape: {features_train.shape}")
print(f"Testing features shape: {features_test.shape}")
print(f"Training labels shape: {labels_train.shape}")
print(f"Testing labels shape: {labels_test.shape}")

Training features shape: (11520, 224, 224, 3)
Testing features shape: (1280, 224, 224, 3)
Training labels shape: (11520, 4)
Testing labels shape: (1280, 4)


#Training

In [39]:
# Import necessary libraries
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [40]:
# Add preprocessing layer at the front of VGG16
vgg = VGG16(input_shape=features_train.shape[1:], weights='imagenet', include_top=False)

# Prevent training already trained layers
for layer in vgg.layers:
    layer.trainable = False

# Add Flatten layer
x = Flatten()(vgg.output)

# Add Dense layers with weight regularization and dropout to reduce overfitting
x = Dense(1000, activation="relu", kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)

x = Dense(256, activation="relu", kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)

# Dense layer with number of neurons equal to number of classes
prediction = Dense(labels_train.shape[1], activation='softmax')(x)

# Create the model object
model = Model(inputs=vgg.input, outputs=prediction)

# Print model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1000)                │      25,089,000 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 40,060,972 (152.82 MB)

 Trainable params: 25,346,284 (96.69 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [26]:
# Import additional necessary libraries
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Set up the optimizer and learning rate scheduler
optimizer = Adam(learning_rate=0.0001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# Create an instance of EarlyStopping callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

# Model checkpoint to save the best weights
checkpoint = ModelCheckpoint("weights.best.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Callbacks list
callbacks_list = [early_stopping_callback, checkpoint, reduce_lr]

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

In [41]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler, TensorBoard
from tensorflow.keras.optimizers import Adam

# Set up the optimizer and learning rate scheduler
optimizer = Adam(learning_rate=0.0001)

# Reduce learning rate with a more aggressive factor and smaller patience
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)

# Early stopping with reduced patience
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

# Model checkpoint to save the best weights, monitoring val_loss
checkpoint = ModelCheckpoint("weights.best.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Optionally add learning rate scheduler for warmup
def lr_scheduler(epoch, lr):
    if epoch < 5:
        return lr * (epoch + 1) / 5  # Gradually warm up the learning rate
    return lr  # After 5 epochs, use the base learning rate

lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# TensorBoard for visualization
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

# Callbacks list
callbacks_list = [early_stopping_callback, checkpoint, reduce_lr, lr_scheduler_callback, tensorboard_callback]

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

In [42]:
# Start training the model
history = model.fit(features_train, labels_train, epochs=50,
                    shuffle=True, validation_split=0.1,
                    callbacks=callbacks_list, batch_size=32)

Epoch 1/50
322/324 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2879 - loss: 3.7311
Epoch 1: val_loss improved from inf to 3.18724, saving model to weights.best.keras
324/324 ━━━━━━━━━━━━━━━━━━━━ 18s 43ms/step - accuracy: 0.2881 - loss: 3.7291 - val_accuracy: 0.3707 - val_loss: 3.1872 - learning_rate: 2.0000e-05
Epoch 2/50
324/324 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3763 - loss: 3.1800
Epoch 2: val_loss improved from 3.18724 to 3.05635, saving model to weights.best.keras
324/324 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.3763 - loss: 3.1799 - val_accuracy: 0.4149 - val_loss: 3.0563 - learning_rate: 8.0000e-06
Epoch 3/50
324/324 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3998 - loss: 3.0518
Epoch 3: val_loss improved from 3.05635 to 2.98939, saving model to weights.best.keras
324/324 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.3998 - loss: 3.0518 - val_accuracy: 0.4340 - val_loss: 2.9894 - learning_rate: 4.8000e-06
Epoch 4/50
324/324 ━━━━━━━━━━━━━━━━━━━━ 0

In [43]:
# Load weights of the model with the best validation accuracy
model.load_weights("weights.best.keras")

In [44]:
# Evaluate the model
model_evaluation_history_base = model.evaluate(features_test, labels_test, batch_size=32)
print("Test Loss:", model_evaluation_history_base[0])
print("Test Accuracy:", model_evaluation_history_base[1])

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5455 - loss: 2.2300
Test Loss: 2.253497362136841
Test Accuracy: 0.534375011920929


In [46]:
# Save the model
model.save("compvis_age_predictor_model.keras")

In [69]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the saved model
model = load_model('compvis_age_predictor_model.keras')

# Define the image preprocessing function
def image_preprocessing(img_path):
    # Read the image
    img = cv2.imread(img_path)

    # Resize the image to 224x224 (as expected by VGG16)
    resized_img = cv2.resize(img, (224, 224))

    # Normalize the image
    normalized_img = resized_img / 255.0

    # Convert image to array with the required shape (224, 224, 3)
    img_array = img_to_array(normalized_img)

    # Expand dimensions to make it compatible with the model input (batch size of 1)
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

# Define the function to map the predicted class index to the age range
def map_class_to_age(class_index):
    # Age ranges for the class indices
    age_ranges = {
        0: "0-20",
        1: "21-40",
        2: "41-60",
        3: "61-80"
    }

    return age_ranges.get(class_index, "Unknown")

# Function to predict the age group of a given image
def predict_age(img_path):
    # Preprocess the image
    preprocessed_image = image_preprocessing(img_path)

    # Predict the class (age group)
    predictions = model.predict(preprocessed_image)

    # Get the index of the class with the highest probability
    predicted_class_index = np.argmax(predictions, axis=1)[0]

    # Map the predicted class index to the corresponding age range
    predicted_age_range = map_class_to_age(predicted_class_index)

    return predicted_age_range

# Example usage: predict the age group of a given image
img_path = '/content/premium_photo-1668806642968-c6bebdab7c4c.jpg'  # Replace with the path to your image
predicted_age = predict_age(img_path)

print(f"Predicted Age Range: {predicted_age}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
Predicted Age Range: 0-20
